In [ ]:

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
import warnings
warnings.filterwarnings("ignore")
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain_community.vectorstores import FAISS
import os
import json
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import defaultdict
import faiss
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import re
from langchain_community.document_loaders import TextLoader
# from langchain.chat_models import ChatOpenAI
# from langchain.chains import load_qa_chain
from langchain.callbacks import get_openai_callback
from collections import defaultdict
import json
from sentence_transformers import SentenceTransformer
# import faiss
from sklearn.metrics.pairwise import cosine_similarity
from langchain.schema import Document



# Initialize Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example of labels you want to use
labels = ["Important Dates", "Eligibility or Prequalification Criteria", "Scope of Work", "Contact Details"]

outFile = r'/data/tendergpt/testing/77774640/77774640.txt'
with open(outFile, 'r', encoding='utf-8') as f:
    essay = f.read()

# Split the essay into sentences
single_sentences_list = re.split(r'(?<=[.?!])\s+', essay)

# Create sentence objects with index
sentences = [{'sentence': x, 'index': i} for i, x in enumerate(single_sentences_list)]

# Function to combine sentences into chunks
def combine_sentences(sentences, buffer_size=1):
    for i in range(len(sentences)):
        combined_sentence = ''
        for j in range(i - buffer_size, i):
            if j >= 0:
                combined_sentence += sentences[j]['sentence'] + ' '

        combined_sentence += sentences[i]['sentence']

        for j in range(i + 1, i + 1 + buffer_size):
            if j < len(sentences):
                combined_sentence += ' ' + sentences[j]['sentence']
        sentences[i]['combined_sentence'] = combined_sentence
    return sentences

# Combine sentences and create embeddings
sentences = combine_sentences(sentences)
print("sentences:::",sentences)
embeddings = model.encode([x['combined_sentence'] for x in sentences])

# Add embeddings to sentences
for i, sentence in enumerate(sentences):
    sentence['combined_sentence_embedding'] = embeddings[i]

# Function to calculate cosine distances between sentence embeddings
def calculate_cosine_distances(sentences):
    distances = []
    for i in range(len(sentences) - 1):
        embedding_current = sentences[i]['combined_sentence_embedding']
        embedding_next = sentences[i + 1]['combined_sentence_embedding']
        similarity = np.dot(embedding_current, embedding_next) / (np.linalg.norm(embedding_current) * np.linalg.norm(embedding_next))
        distance = 1 - similarity
        distances.append(distance)
        sentences[i]['distance_to_next'] = distance
    return distances, sentences

# Calculate distances between sentences
distances, sentences = calculate_cosine_distances(sentences)

# Define a threshold for splitting chunks
breakpoint_percentile_threshold = 95
breakpoint_distance_threshold = np.percentile(distances, breakpoint_percentile_threshold)
indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold]

# Split the document into chunks based on distance threshold
start_index = 0
chunks = []
for index in indices_above_thresh:
    end_index = index
    group = sentences[start_index:end_index + 1]
    combined_text = ' '.join([d['sentence'] for d in group])
    chunks.append(combined_text)
    start_index = index + 1

if start_index < len(sentences):    
    combined_text = ' '.join([d['sentence'] for d in sentences[start_index:]])
    chunks.append(combined_text)

# Function to chunk large text into smaller parts based on token count
def chunk_text(text, max_tokens=3500):
    max_chars = max_tokens * 2  # Approximate token-to-character conversion
    chunks = [text[i:i + max_chars] for i in range(0, len(text), max_chars)]
    return chunks

# Combine the document chunks into a single large text if needed
large_text = " ".join(chunks)
print("large_text:::",large_text)
# Split large text into smaller chunks that fit within the token limit
chunked_texts = chunk_text(large_text)
print("chunked_texts:::",chunked_texts)
# Initialize the language model
llm = ChatOpenAI(
    model_name="meta-llama/Llama-3.1-8B-Instruct",
    openai_api_base="http://localhost:8000/v1",
    openai_api_key="FAKE",  # Replace with your actual key if needed
    max_tokens=500,  # Adjust this based on your use case
    temperature=0.1
)

# Load QA chain
chain = load_qa_chain(llm, verbose=True, chain_type='stuff')

# Function to process chunks and run LLM
def paramExtractionUsingLLM(chunks, label=""):
    print(" -- inside llm -- ")
    all_results = []
    max_chars = 23520  # Token size limit per chunk
    finalText = ""
    for c in chunks:
        finalText += c
    print("Total length of chunk: ", len(finalText))

    if len(finalText) > max_chars:
        split_texts = [finalText[i:i + max_chars] for i in range(0, len(finalText), max_chars)]
    else:
        split_texts = None

    results = []
    if split_texts:
        print(" -- inside if -- ")
        for text in split_texts:
            print("-- in for loop --")
            f"""Extract only the relevant details for the label '{label}' from the below text.
Do not include any irrelevant information or summaries. Provide a direct response to the question.

Text: {text}

Output (only relevant details): 
"""
            try:
                with get_openai_callback() as cost:
                    response = chain.run(input_documents=[Document(page_content=text)], question=label)
                results.append(response.strip())
            except Exception as e:
                print("Error:", e)
                results.append("Error processing chunk")
        
        combined_result = " ".join(results)
        all_results.append(combined_result)
    else:
        print(" -- inside else -- ")
        f"""Extract only the relevant details for the label '{label}' from the below text.
Do not include any irrelevant information or summaries. Provide a direct response to the question.

Text: {text}

Output (only relevant details): 
"""
        try:
            with get_openai_callback() as cost:
                response = chain.run(input_documents=[Document(page_content=finalText)], question=label)
            results.append(response.strip())
        except Exception as e:
            print("Error:", e)
            results.append("Error processing chunk")
    
        combined_result = " ".join(results)
        all_results.append(combined_result)
    
    return all_results

# Main function to process all labels
def main(label):
    eligibility_criteria_chunk = chunks  # Adjust according to the chunking
    eligibility_results = paramExtractionUsingLLM(eligibility_criteria_chunk, label=label)
    print(" -- LLM Response -- ")
    print(eligibility_results)
    return eligibility_results

# Create a dictionary to hold the results for all labels
labeled_ans = defaultdict(list)
for label in labels:
    result = main(label)
    labeled_ans[label].append(result)

# Save LLM output
ll_response_out_file_path = r'/data/QAAPI/stored_files/lllm_out.json'
with open(ll_response_out_file_path, 'w', encoding='utf-8') as out_file:
    json.dump(labeled_ans, out_file, indent=4, ensure_ascii=False)


In [1]:
import os

def count_subfolders(main_folder_path):
    return len([f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))])

# Example usage
main_folder ="/data/tendergpt/livetender_txt"
# main_folder = "/data/dailydocument"
# main_folder = "/data/tendergpt/livetender_txt"
# main_folder= "/data/tendergpt/03_txt_testing"
# main_folder = "/data/txtfolder/dailydocument_29-11-24_txt"
# main_folder = "/data/unzipdocument/dailydocument_02-12-24"



print("Number of subfolders:", count_subfolders(main_folder))


Number of subfolders: 1935875


In [ ]:
import os
import re
import json
import logging
import warnings
from typing import List, Dict, Any

import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_distances

# Langchain imports
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain.chains.question_answering import load_qa_chain

# Configure logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.FileHandler('/data/tendergpt/processing_log.txt'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Suppress warnings
warnings.filterwarnings("ignore")

class TenderDocumentProcessor:
    def __init__(self, config: Dict[str, Any]):
        """
        Initialize the Tender Document Processor
        
        :param config: Configuration dictionary with processing parameters
        """
        self.config = config
        
        # Initialize models
        try:
            self.embedding_model = SentenceTransformer(config['embedding_model'])
            self.llm = ChatOpenAI(
                model_name=config['llm_model'],
                openai_api_base=config['api_base'],
                openai_api_key=config['api_key'],
                max_tokens=config['max_tokens'],
                temperature=config['temperature']
            )
            self.qa_chain = load_qa_chain(self.llm, chain_type='stuff', verbose=False)
        except Exception as e:
            logger.error(f"Model initialization error: {e}")
            raise

    def load_document(self, file_path: str) -> str:
        """
        Load document content safely
        
        :param file_path: Path to the input file
        :return: Document text
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        except IOError as e:
            logger.error(f"Error reading file {file_path}: {e}")
            return ""

    def preprocess_text(self, text: str) -> List[str]:
        """
        Preprocess text into sentences
        
        :param text: Input document text
        :return: List of cleaned sentences
        """
        # Use more robust sentence splitting
        sentences = re.split(r'(?<=[.!?])\s+', text)
        return [sent.strip() for sent in sentences if sent.strip()]

    def generate_embeddings(self, sentences: List[str]) -> np.ndarray:
        """
        Generate sentence embeddings
        
        :param sentences: List of sentences
        :return: Numpy array of embeddings
        """
        try:
            return self.embedding_model.encode(sentences, show_progress_bar=False)
        except Exception as e:
            logger.error(f"Embedding generation error: {e}")
            return np.array([])

    def calculate_sentence_distances(self, embeddings: np.ndarray) -> np.ndarray:
        """
        Calculate pairwise cosine distances between sentence embeddings
        
        :param embeddings: Numpy array of sentence embeddings
        :return: Array of pairwise distances
        """
        try:
            # Compute cosine distances
            distances = cosine_distances(embeddings)
            
            # Extract distances between consecutive sentences
            # Add padding to handle edge cases
            pairwise_distances = np.diagonal(distances, offset=1)
            pairwise_distances = np.pad(
                pairwise_distances, 
                (0, 1), 
                mode='constant', 
                constant_values=0
            )
            
            return pairwise_distances
        except Exception as e:
            logger.error(f"Distance calculation error: {e}")
            return np.array([])

    def chunk_document(
        self, 
        sentences: List[str], 
        distances: np.ndarray, 
        threshold_percentile: float = 95
    ) -> List[str]:
        """
        Chunk document based on semantic distances
        
        :param sentences: List of sentences
        :param distances: Pairwise sentence distances
        :param threshold_percentile: Percentile for chunk breaking
        :return: List of document chunks
        """
        # Handle edge cases
        if not sentences or len(distances) == 0:
            return [' '.join(sentences)]

        # Calculate distance threshold
        try:
            distance_threshold = np.percentile(distances, threshold_percentile)
        except Exception as e:
            logger.warning(f"Percentile calculation error, using default: {e}")
            distance_threshold = np.max(distances) * 0.8

        chunks = []
        current_chunk = []

        for sentence, distance in zip(sentences, distances):
            current_chunk.append(sentence)
            
            # Break chunk if distance exceeds threshold
            if distance > distance_threshold:
                chunks.append(' '.join(current_chunk))
                current_chunk = []

        # Add remaining sentences
        if current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks

    def extract_information(self, chunk: str, label: str) -> str:
        """
        Extract specific information using LLM
        
        :param chunk: Text chunk to process
        :param label: Information label to extract
        :return: Extracted information
        """
        try:
            prompt = f"""Extract only the relevant details for the label '{label}' from the text.
Do not include irrelevant information or summaries. Provide a direct, concise response.

Text: {chunk}

Output (only relevant details):"""
            
            # Use QA chain for extraction
            response = self.qa_chain.run(
                input_documents=[Document(page_content=chunk)], 
                question=prompt
            )
            
            return response.strip()
        except Exception as e:
            logger.error(f"Information extraction error for {label}: {e}")
            return "Could not extract information"

    def process_document(self) -> Dict[str, List[str]]:
        """
        Main document processing method
        
        :return: Dictionary of extracted information by label
        """
        # Load document
        document_text = self.load_document(self.config['input_file'])
        if not document_text:
            logger.error("Empty document. Processing cannot continue.")
            return {}

        # Preprocess sentences
        sentences = self.preprocess_text(document_text)
        
        # Generate embeddings
        embeddings = self.generate_embeddings(sentences)
        if len(embeddings) == 0:
            logger.error("Embedding generation failed.")
            return {}

        # Calculate distances
        distances = self.calculate_sentence_distances(embeddings)
        
        # Chunk document
        chunks = self.chunk_document(
            sentences, 
            distances, 
            self.config['distance_percentile']
        )
        
        # Extract information for each label
        labeled_results = {}
        for label in self.config['labels']:
            results = [
                self.extract_information(chunk, label) 
                for chunk in chunks
            ]
            # Filter out empty results
            labeled_results[label] = [r for r in results if r]

        return labeled_results

    def save_results(self, results: Dict[str, List[str]]):
        """
        Save processing results to a JSON file
        
        :param results: Extracted information dictionary
        """
        try:
            with open(self.config['output_file'], 'w', encoding='utf-8') as out_file:
                json.dump(results, out_file, indent=4, ensure_ascii=False)
            logger.info(f"Results saved to {self.config['output_file']}")
        except Exception as e:
            logger.error(f"Error saving results: {e}")

def main():
    # Centralized configuration
    CONFIG = {
        'embedding_model': 'all-MiniLM-L6-v2',
        'llm_model': 'meta-llama/Llama-3.1-8B-Instruct',
        'api_base': 'http://localhost:8000/v1',
        'api_key': 'FAKE',
        'max_tokens': 500,
        'temperature': 0.1,
        'labels': [
            "Important Dates", 
            "Eligibility or Prequalification Criteria", 
            "Scope of Work", 
            "Contact Details"
        ],
        'input_file': '/data/tendergpt/testing/77408673/77408673.txt',
        'output_file': '/data/QAAPI/stored_files/lllm_out.json',
        'distance_percentile': 95
    }

    try:
        # Initialize and process document
        processor = TenderDocumentProcessor(CONFIG)
        results = processor.process_document()
        
        # Save results
        processor.save_results(results)
        
        logger.info("Document processing completed successfully.")
    
    except Exception as e:
        logger.error(f"Critical error in document processing: {e}")

if __name__ == "__main__":
    main()

## LLMCHAIN 

In [ ]:

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
import warnings
warnings.filterwarnings("ignore")
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain_community.vectorstores import FAISS
import os
import json
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import defaultdict
import faiss
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import re
from langchain_community.document_loaders import TextLoader
# from langchain.chat_models import ChatOpenAI
# from langchain.chains import load_qa_chain
from langchain.callbacks import get_openai_callback
from collections import defaultdict
import json
from sentence_transformers import SentenceTransformer
# import faiss
from sklearn.metrics.pairwise import cosine_similarity
from langchain.schema import Document



# Initialize Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example of labels you want to use
labels = ["Important Dates", "Eligibility or Prequalification Criteria", "Scope of Work", "Contact Details"]

outFile = r'/data/tendergpt/testing/77408673/77408673.txt'
with open(outFile, 'r', encoding='utf-8') as f:
    essay = f.read()

# Split the essay into sentences
single_sentences_list = re.split(r'(?<=[.?!])\s+', essay)

# Create sentence objects with index
sentences = [{'sentence': x, 'index': i} for i, x in enumerate(single_sentences_list)]

# Function to combine sentences into chunks
def combine_sentences(sentences, buffer_size=1):
    for i in range(len(sentences)):
        combined_sentence = ''
        for j in range(i - buffer_size, i):
            if j >= 0:
                combined_sentence += sentences[j]['sentence'] + ' '

        combined_sentence += sentences[i]['sentence']

        for j in range(i + 1, i + 1 + buffer_size):
            if j < len(sentences):
                combined_sentence += ' ' + sentences[j]['sentence']
        sentences[i]['combined_sentence'] = combined_sentence
    return sentences

# Combine sentences and create embeddings
sentences = combine_sentences(sentences)
embeddings = model.encode([x['combined_sentence'] for x in sentences])

# Add embeddings to sentences
for i, sentence in enumerate(sentences):
    sentence['combined_sentence_embedding'] = embeddings[i]

# Function to calculate cosine distances between sentence embeddings
def calculate_cosine_distances(sentences):
    distances = []
    for i in range(len(sentences) - 1):
        embedding_current = sentences[i]['combined_sentence_embedding']
        embedding_next = sentences[i + 1]['combined_sentence_embedding']
        similarity = np.dot(embedding_current, embedding_next) / (np.linalg.norm(embedding_current) * np.linalg.norm(embedding_next))
        distance = 1 - similarity
        distances.append(distance)
        sentences[i]['distance_to_next'] = distance
    return distances, sentences

# Calculate distances between sentences
distances, sentences = calculate_cosine_distances(sentences)

# Define a threshold for splitting chunks
breakpoint_percentile_threshold = 95
breakpoint_distance_threshold = np.percentile(distances, breakpoint_percentile_threshold)
indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold]

# Split the document into chunks based on distance threshold
start_index = 0
chunks = []
for index in indices_above_thresh:
    end_index = index
    group = sentences[start_index:end_index + 1]
    combined_text = ' '.join([d['sentence'] for d in group])
    chunks.append(combined_text)
    start_index = index + 1

if start_index < len(sentences):    
    combined_text = ' '.join([d['sentence'] for d in sentences[start_index:]])
    chunks.append(combined_text)

# Function to chunk large text into smaller parts based on token count
def chunk_text(text, max_tokens=3500):
    max_chars = max_tokens * 2  # Approximate token-to-character conversion
    chunks = [text[i:i + max_chars] for i in range(0, len(text), max_chars)]
    return chunks

# Combine the document chunks into a single large text if needed
large_text = " ".join(chunks)

# Split large text into smaller chunks that fit within the token limit
chunked_texts = chunk_text(large_text)
# Initialize the language model
llm = ChatOpenAI(
    model_name="meta-llama/Llama-3.1-8B-Instruct",
    openai_api_base="http://localhost:8000/v1",
    openai_api_key="FAKE",  # Replace with your actual key if needed
    max_tokens=500,  # Adjust this based on your use case
    temperature=0.1
)

# Define a prompt template for the chain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_template = PromptTemplate(
    input_variables=["text", "label"],
    template=(
        "Extract only the relevant details for the label '{label}' from the text below.\n"
        "Do not include any irrelevant information or summaries. Provide a direct response to the question.\n\n"
        "Text: {text}\n\n"
        "Output (only relevant details):"
    )
)

# Create the LLM chain using the prompt and the Llama 3 model
chain = LLMChain(llm=llm, verbose=True, prompt=prompt_template)

# Function to process chunks and run LLM using LLMChain
def paramExtractionUsingLLM(chunks, label=""):
    print(" -- inside llm -- ")
    all_results = []
    max_chars = 23520  # Token size limit per chunk

    final_text = " ".join(chunks)
    print("Total length of chunk: ", len(final_text))

    if len(final_text) > max_chars:
        split_texts = [final_text[i:i + max_chars] for i in range(0, len(final_text), max_chars)]
    else:
        split_texts = [final_text]

    results = []
    for text in split_texts:
        try:
            response = chain.run(text=text, label=label)
            results.append(response.strip())
        except Exception as e:
            print("Error:", e)
            results.append("Error processing chunk")
        
    combined_result = " ".join(results)
    all_results.append(combined_result)
    
    return all_results

# Main function to process all labels
def main(label):
    eligibility_criteria_chunk = chunks  # Adjust according to the chunking
    eligibility_results = paramExtractionUsingLLM(eligibility_criteria_chunk, label=label)
    print(" -- LLM Response -- ")
    print(eligibility_results)
    return eligibility_results

# Create a dictionary to hold the results for all labels
labeled_ans = defaultdict(list)
for label in labels:
    result = main(label)
    labeled_ans[label].append(result)

# Save LLM output
ll_response_out_file_path = r'/data/QAAPI/stored_files/lllm2_out.json'
with open(ll_response_out_file_path, 'w', encoding='utf-8') as out_file:
    json.dump(labeled_ans, out_file, indent=4, ensure_ascii=False)



## set dynamic chunking

In [ ]:
import os
import re
import json
import warnings
import numpy as np
from collections import defaultdict

# Suppress warnings
warnings.filterwarnings("ignore")

# Embedding and ML libraries
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# LangChain libraries
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback

class SemanticChunker:
    def __init__(self, model_name='all-MiniLM-L6-v2', 
                 breakpoint_percentile=95, 
                 buffer_size=1):
        """
        Initialize Semantic Chunker with embedding model and chunking parameters
        
        Args:
            model_name (str): Sentence transformer model name
            breakpoint_percentile (int): Percentile for distance threshold
            buffer_size (int): Number of surrounding sentences to include in context
        """
        self.model = SentenceTransformer(model_name)
        self.breakpoint_percentile = breakpoint_percentile
        self.buffer_size = buffer_size

    def preprocess_text(self, text):
        """
        Preprocess text by splitting into sentences
        
        Args:
            text (str): Input text
        
        Returns:
            list: List of sentences with indices
        """
        sentences = re.split(r'(?<=[.?!])\s+', text)
        return [{'sentence': x, 'index': i} for i, x in enumerate(sentences)]

    def combine_sentences(self, sentences):
        """
        Combine sentences with surrounding context
        
        Args:
            sentences (list): List of sentence dictionaries
        
        Returns:
            list: Enhanced sentence dictionaries with combined context
        """
        for i in range(len(sentences)):
            combined_sentence = ''
            
            # Add preceding context
            for j in range(max(0, i - self.buffer_size), i):
                combined_sentence += sentences[j]['sentence'] + ' '
            
            # Add current sentence
            combined_sentence += sentences[i]['sentence']
            
            # Add following context
            for j in range(i + 1, min(len(sentences), i + 1 + self.buffer_size)):
                combined_sentence += ' ' + sentences[j]['sentence']
            
            sentences[i]['combined_sentence'] = combined_sentence
        return sentences

    def calculate_cosine_distances(self, sentences):
        """
        Calculate cosine distances between sentence embeddings
        
        Args:
            sentences (list): List of sentence dictionaries
        
        Returns:
            tuple: Distances and updated sentences
        """
        # Encode combined sentences
        embeddings = self.model.encode([x['combined_sentence'] for x in sentences])
        
        # Add embeddings to sentences
        for i, sentence in enumerate(sentences):
            sentence['combined_sentence_embedding'] = embeddings[i]
        
        # Calculate inter-sentence distances
        distances = []
        for i in range(len(sentences) - 1):
            embedding_current = sentences[i]['combined_sentence_embedding']
            embedding_next = sentences[i + 1]['combined_sentence_embedding']
            
            similarity = np.dot(embedding_current, embedding_next) / (
                np.linalg.norm(embedding_current) * np.linalg.norm(embedding_next)
            )
            distance = 1 - similarity
            distances.append(distance)
            sentences[i]['distance_to_next'] = distance
        
        return distances, sentences

    def chunk_text(self, text, max_tokens=3500):
        """
        Dynamic chunking based on semantic distances
        
        Args:
            text (str): Input text
            max_tokens (int): Maximum tokens per chunk
        
        Returns:
            list: Semantic chunks
        """
        # Preprocess and combine sentences
        sentences = self.preprocess_text(text)
        sentences = self.combine_sentences(sentences)
        
        # Calculate cosine distances
        distances, sentences = self.calculate_cosine_distances(sentences)
        
        # Determine breakpoint threshold
        breakpoint_distance_threshold = np.percentile(distances, self.breakpoint_percentile)
        
        # Split document into chunks
        chunks = []
        start_index = 0
        indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold]
        
        for index in indices_above_thresh:
            end_index = index
            group = sentences[start_index:end_index + 1]
            combined_text = ' '.join([d['sentence'] for d in group])
            chunks.append(combined_text)
            start_index = index + 1
        
        # Add remaining text
        if start_index < len(sentences):    
            combined_text = ' '.join([d['sentence'] for d in sentences[start_index:]])
            chunks.append(combined_text)
        
        return chunks

def paramExtractionUsingLLM(chunks, label, llm, chain):
    """
    Extract parameters from text chunks using Language Model
    
    Args:
        chunks (list): Text chunks
        label (str): Label for extraction
        llm: Language model
        chain: QA chain
    
    Returns:
        list: Extracted results
    """
    all_results = []
    max_chars = 23520  # Token size limit per chunk

    finalText = " ".join(chunks)
    
    if len(finalText) > max_chars:
        split_texts = [finalText[i:i + max_chars] for i in range(0, len(finalText), max_chars)]
    else:
        split_texts = [finalText]

    results = []
    for text in split_texts:
        prompt = f"""Extract only the relevant details for the label '{label}' 
        from the text. Do not include irrelevant information or summaries.

        Text: {text}

        Output (relevant details): """
        
        try:
            with get_openai_callback() as cost:
                response = chain.run(input_documents=[Document(page_content=text)], question=prompt)
            results.append(response.strip())
        except Exception as e:
            print(f"Error processing chunk for {label}: {e}")
            results.append("Error processing chunk")
    
    return [" ".join(results)]

def main(file_path, labels):
    """
    Main processing function
    
    Args:
        file_path (str): Path to input text file
        labels (list): Labels for extraction
    
    Returns:
        dict: Extracted labeled information
    """
    # Read input text
    with open(file_path, 'r', encoding='utf-8') as f:
        essay = f.read()
    
    # Initialize semantic chunker
    chunker = SemanticChunker()
    chunks = chunker.chunk_text(essay)
    
    # Initialize LLM
    llm = ChatOpenAI(
        model_name="meta-llama/Llama-3.1-8B-Instruct",
        openai_api_base="http://localhost:8000/v1",
        openai_api_key="FAKE",
        max_tokens=1000,
        temperature=0.1
    )
    
    # Load QA chain
    chain = load_qa_chain(llm, verbose=True, chain_type='stuff')
    
    # Process labels
    labeled_ans = defaultdict(list)
    for label in labels:
        result = paramExtractionUsingLLM(chunks, label, llm, chain)
        labeled_ans[label].append(result)
    
    return labeled_ans

# Example usage
labels = ["Important Dates", "Eligibility Criteria", "Scope of Work", "Contact Details"]
input_file = '/data/tendergpt/testing/77774640/77774640.txt'
output_file = '/data/QAAPI/stored_files/llm_out.json'

# Run extraction
results = main(input_file, labels)

# Save results
with open(output_file, 'w', encoding='utf-8') as out_file:
    json.dump(results, out_file, indent=4, ensure_ascii=False)

## error with extract max_model_length

In [ ]:
import os
import re
import json
import warnings
import numpy as np
from collections import defaultdict

# Suppress warnings
warnings.filterwarnings("ignore")

# Embedding and ML libraries
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# LangChain libraries
from langchain.docstore.document import Document
from langchain.text_splitter import TokenTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback

class SemanticChunker:
    def __init__(self, model_name='all-MiniLM-L6-v2', 
                 breakpoint_percentile=95, 
                 buffer_size=1,
                 max_tokens=7000):
        """
        Initialize Semantic Chunker with embedding model and chunking parameters
        
        Args:
            model_name (str): Sentence transformer model name
            breakpoint_percentile (int): Percentile for distance threshold
            buffer_size (int): Number of surrounding sentences to include in context
            max_tokens (int): Maximum tokens per chunk
        """
        self.model = SentenceTransformer(model_name)
        self.breakpoint_percentile = breakpoint_percentile
        self.buffer_size = buffer_size
        self.max_tokens = max_tokens
        self.token_splitter = TokenTextSplitter(chunk_size=max_tokens, chunk_overlap=200)

    def preprocess_text(self, text):
        """
        Preprocess text by splitting into sentences
        
        Args:
            text (str): Input text
        
        Returns:
            list: List of sentences with indices
        """
        sentences = re.split(r'(?<=[.?!])\s+', text)
        return [{'sentence': x, 'index': i} for i, x in enumerate(sentences) if x.strip()]

    def combine_sentences(self, sentences):
        """
        Combine sentences with surrounding context
        
        Args:
            sentences (list): List of sentence dictionaries
        
        Returns:
            list: Enhanced sentence dictionaries with combined context
        """
        for i in range(len(sentences)):
            combined_sentence = ''
            
            # Add preceding context
            for j in range(max(0, i - self.buffer_size), i):
                combined_sentence += sentences[j]['sentence'] + ' '
            
            # Add current sentence
            combined_sentence += sentences[i]['sentence']
            
            # Add following context
            for j in range(i + 1, min(len(sentences), i + 1 + self.buffer_size)):
                combined_sentence += ' ' + sentences[j]['sentence']
            
            sentences[i]['combined_sentence'] = combined_sentence
        return sentences

    def calculate_cosine_distances(self, sentences):
        """
        Calculate cosine distances between sentence embeddings
        
        Args:
            sentences (list): List of sentence dictionaries
        
        Returns:
            tuple: Distances and updated sentences
        """
        # Encode combined sentences
        embeddings = self.model.encode([x['combined_sentence'] for x in sentences])
        
        # Add embeddings to sentences
        for i, sentence in enumerate(sentences):
            sentence['combined_sentence_embedding'] = embeddings[i]
        
        # Calculate inter-sentence distances
        distances = []
        for i in range(len(sentences) - 1):
            embedding_current = sentences[i]['combined_sentence_embedding']
            embedding_next = sentences[i + 1]['combined_sentence_embedding']
            
            similarity = np.dot(embedding_current, embedding_next) / (
                np.linalg.norm(embedding_current) * np.linalg.norm(embedding_next)
            )
            distance = 1 - similarity
            distances.append(distance)
            sentences[i]['distance_to_next'] = distance
        
        return distances, sentences

    def chunk_text(self, text):
        """
        Dynamic chunking based on semantic distances and token management
        
        Args:
            text (str): Input text
        
        Returns:
            list: Semantic chunks
        """
        # First, use token-based splitting as a fallback
        if len(text) > self.max_tokens * 2:  # Rough token estimation
            return self.token_splitter.split_text(text)
        
        # Preprocess and combine sentences
        sentences = self.preprocess_text(text)
        sentences = self.combine_sentences(sentences)
        
        # Calculate cosine distances
        distances, sentences = self.calculate_cosine_distances(sentences)
        
        # Determine breakpoint threshold
        breakpoint_distance_threshold = np.percentile(distances, self.breakpoint_percentile)
        
        # Split document into chunks
        chunks = []
        start_index = 0
        indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold]
        
        for index in indices_above_thresh:
            end_index = index
            group = sentences[start_index:end_index + 1]
            combined_text = ' '.join([d['sentence'] for d in group])
            chunks.append(combined_text)
            start_index = index + 1
        
        # Add remaining text
        if start_index < len(sentences):    
            combined_text = ' '.join([d['sentence'] for d in sentences[start_index:]])
            chunks.append(combined_text)
        
        # Fallback to token-based splitting if chunks are too large
        final_chunks = []
        for chunk in chunks:
            if len(chunk) > self.max_tokens * 2:
                final_chunks.extend(self.token_splitter.split_text(chunk))
            else:
                final_chunks.append(chunk)
        
        return final_chunks

def paramExtractionUsingLLM(chunks, label, llm, chain):
    """
    Extract parameters from text chunks using Language Model
    
    Args:
        chunks (list): Text chunks
        label (str): Label for extraction
        llm: Language model
        chain: QA chain
    
    Returns:
        list: Extracted results
    """
    results = []
    for chunk in chunks:
        prompt = f"""Extract only the precise and most relevant details for the label '{label}'. 
        Focus on key information. Avoid redundancy and summaries.

        Context: {chunk}

        Extraction: """
        
        try:
            with get_openai_callback() as cost:
                response = chain.run(input_documents=[Document(page_content=chunk)], question=prompt)
            
            # Only append non-empty results
            if response and response.strip():
                results.append(response.strip())
        except Exception as e:
            print(f"Error processing chunk for {label}: {e}")
    
    return [" ".join(results)] if results else ["No relevant information found"]

def main(file_path, labels):
    """
    Main processing function
    
    Args:
        file_path (str): Path to input text file
        labels (list): Labels for extraction
    
    Returns:
        dict: Extracted labeled information
    """
    # Read input text
    with open(file_path, 'r', encoding='utf-8') as f:
        essay = f.read()
    
    # Initialize semantic chunker
    chunker = SemanticChunker(max_tokens=7000)
    chunks = chunker.chunk_text(essay)
    
    # Initialize LLM
    llm = ChatOpenAI(
        model_name="meta-llama/Llama-3.1-8B-Instruct",
        openai_api_base="http://localhost:8000/v1",
        openai_api_key="FAKE",
        max_tokens=700,  # Reduced max tokens
        temperature=0.1
    )
    
    # Load QA chain
    chain = load_qa_chain(llm, verbose=True, chain_type='stuff')
    
    # Process labels
    labeled_ans = defaultdict(list)
    for label in labels:
        result = paramExtractionUsingLLM(chunks, label, llm, chain)
        labeled_ans[label].append(result)
    
    return labeled_ans

# Example usage
labels = ["Important Dates", "Eligibility Criteria", "Scope of Work", "Contact Details"]
input_file = '/data/tendergpt/testing/77774640/77774640.txt'
output_file = '/data/QAAPI/stored_files/llm_out.json'

# Run extraction
results = main(input_file, labels)

# Save results
with open(output_file, 'w', encoding='utf-8') as out_file:
    json.dump(results, out_file, indent=4, ensure_ascii=False)